In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("input.csv.zip")
df_2 = pd.read_csv("input_2Q.csv.zip")
df_4 = pd.read_csv("input_4Q.csv.zip")

In [ ]:
df_cusip = df.CUSIP.unique().tolist()
df2_cusip = df_2.cusip.unique().tolist()
df4_cusip = df_4.cusip.unique().tolist()


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

list_1q = list(chunks(df_cusip,20))
list_2q = list(chunks(df2_cusip,20))
list_4q = list(chunks(df4_cusip,20))

In [ ]:
import eikon as ek
ek.set_app_key("03edf78fe41c4945af6db249d605d0ee35ff7931")

In [ ]:
fields = ['TR.NumberOfAnalysts.calcdate','TR.NumberOfAnalysts.periodenddate','TR.NumberOfAnalysts']
params = {'SDate': "1994-12-31", 'EDate': "2020-12-31", 'FRQ': 'CQ',"Period":"FQ2"}

df_temp,err_temp = ek.get_data("037833100",fields,params)

# 1Q

In [ ]:
import time
df1_final_list = []
i=0
fields = ['TR.NumberOfAnalysts.calcdate','TR.NumberOfAnalysts.periodenddate','TR.NumberOfAnalysts']
params = {'SDate': "1994-12-31", 'EDate': "2020-12-31", 'FRQ': 'CQ',"Period":"FQ2"}

for li in list_1q:
    df_temp,err_temp = ek.get_data(li,fields,params)
    df1_final_list.append(df_temp)
    time.sleep(1)
    i +=1
    print(i)
df_anal_cov_1Q= pd.concat(df1_final_list, ignore_index=True)

In [ ]:
df_anal_cov_1Q.groupby("Instrument").std()

# 2Q

In [ ]:
import time
df2_final_list = []
i=0
fields = ['TR.NumberOfAnalysts.calcdate','TR.NumberOfAnalysts.periodenddate','TR.NumberOfAnalysts']
params = {'SDate': "1994-12-31", 'EDate': "2020-12-31", 'FRQ': 'CQ',"Period":"FQ3"}

for li in list_2q:
    df_temp,err_temp = ek.get_data(li,fields,params)
    df2_final_list.append(df_temp)
    time.sleep(1)
    i +=1
    print(i)
df_anal_cov_2Q= pd.concat(df2_final_list, ignore_index=True)

In [ ]:
df_anal_cov_2Q

# 4Q

In [ ]:
import time
df4_final_list = []
i=0
fields = ['TR.NumberOfAnalysts.calcdate','TR.NumberOfAnalysts.periodenddate','TR.NumberOfAnalysts']
params = {'SDate': "1994-12-31", 'EDate': "2020-12-31", 'FRQ': 'CQ',"Period":"FQ5"}

for li in list_4q:
    df_temp,err_temp = ek.get_data(li,fields,params)
    df4_final_list.append(df_temp)
    time.sleep(1)
    i +=1
    print(i)
df_anal_cov_4Q= pd.concat(df4_final_list, ignore_index=True)

In [ ]:
df_anal_cov_4Q

In [ ]:
df_anal_cov_4Q[df_anal_cov_4Q["Number of Analysts"]==0]

In [ ]:
df_anal_cov_1Q.to_csv("df_anal_cov_1Q.csv",index=False)
df_anal_cov_2Q.to_csv("df_anal_cov_2Q.csv",index=False)
df_anal_cov_4Q.to_csv("df_anal_cov_4Q.csv",index=False)

In [ ]:
test = pd.read_csv("df_anal_cov_2Q.csv")

# merging

In [ ]:
df["Forecasttidspunkt"] = pd.to_datetime(df["Forecasttidspunkt"])
df_anal_cov_1Q["date"] =  pd.to_datetime(df_anal_cov_1Q["Calc Date"])

df_2["Forecasttidspunkt"] = pd.to_datetime(df_2["Forecasttidspunkt"])
df_anal_cov_2Q["date"] =  pd.to_datetime(df_anal_cov_2Q["Calc Date"])

df_4["Forecasttidspunkt"] = pd.to_datetime(df_4["Forecasttidspunkt"])
df_anal_cov_4Q["date"] =  pd.to_datetime(df_anal_cov_4Q["Calc Date"])


In [ ]:
dro

In [ ]:
drop_list = ["Calc Date","Period End Date"]
df_anal_cov_1Q.drop(drop_list,axis=1)

In [ ]:
df_1_merge = df.merge(df_anal_cov_1Q,how="left",left_on=["Forecasttidspunkt","CUSIP"],right_on=["date","Instrument"])

df_2_merge = df_2.merge(df_anal_cov_2Q,how="left",left_on=["Forecasttidspunkt","cusip"],right_on=["date","Instrument"])

df_4_merge = df_4.merge(df_anal_cov_4Q,how="left",left_on=["Forecasttidspunkt","cusip"],right_on=["date","Instrument"])

In [ ]:
df_1_merge["Number of Analysts"].mean()

In [ ]:
df_2_merge["Number of Analysts"].mean()

In [ ]:
df_4_merge["Number of Analysts"].mean()